In [360]:
import pandas as pd
from glob import glob
import re
import os


In [361]:
# 全行全列表示
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# 小数点以下n桁表示
pd.options.display.float_format = '{:.1f}'.format

In [362]:
csv_files = glob('../csv/EXA_FIRST_マイジャグラーV_*.csv')
len(csv_files), csv_files[0]

(152, '../csv\\EXA_FIRST_マイジャグラーV_2024-08-29.csv')

In [363]:
def extract_information(csv_file):
    pattern = r"(.+)_(.+)_(\d{4}-\d{2}-\d{2}).csv"

    file_name = os.path.basename(csv_file)
    # print(file_name)
    match = re.match(pattern, file_name)

    if match:
        hall_name, machine_name, date_str = match.groups()
        return hall_name, machine_name, date_str
    else:
        print(f"ファイル名の形式が一致しません {file_name}")
        print(f"ホール名: {hall_name}")
        print(f"機種名: {machine_name}")
        print(f"日付: {date_str}")


In [364]:
dataframes = []
for csv_file in csv_files:
    hall_name, machine_name, date_str = extract_information(csv_file)
    df = pd.read_csv(csv_file, encoding="utf-8-sig")
    df["date"] = date_str
    df["hall_name"] = hall_name
    df["machine_name"] = machine_name
    dataframes.append(df)

if dataframes:
    df = pd.concat(dataframes, ignore_index=True)
    df.to_csv('../analysis_data.csv', index=False)
df.shape


(12312, 11)

In [365]:
df["G数"] = (
    df["G数"]
    .astype(str)  # 文字列型に変換
    .str.replace(",", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["差枚"] = (
    df["差枚"]
    .astype(str)  # 文字列型に変換
    .str.replace(",", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["合成確率"] = (
    df["合成確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(float)  # 整数型に変換
)
df["BB確率"] = (
    df["BB確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(float)  # 整数型に変換
)
df["RB確率"] = (
    df["RB確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(float)  # 整数型に変換
)

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["month"] = df["date"].dt.month.astype(int)
df['weekday'] = df['date'].dt.weekday.astype(int)
df['day'] = df['date'].dt.day.astype(int)
df["last_digit_of_day"] = df["date"].dt.day.astype(str).str[-1].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12312 entries, 0 to 12311
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   台番号                12312 non-null  int64         
 1   G数                 12312 non-null  int64         
 2   差枚                 12312 non-null  int64         
 3   BB                 12312 non-null  int64         
 4   RB                 12231 non-null  float64       
 5   合成確率               12312 non-null  float64       
 6   BB確率               12312 non-null  float64       
 7   RB確率               12231 non-null  float64       
 8   date               12312 non-null  datetime64[ns]
 9   hall_name          12312 non-null  object        
 10  machine_name       12312 non-null  object        
 11  month              12312 non-null  int64         
 12  weekday            12312 non-null  int64         
 13  day                12312 non-null  int64         
 14  last_d

In [366]:
target_df = df[(df['G数'] >= 2000) & (df['date'] >= pd.to_datetime("2025-01-01"))]
target_df = df[df['G数'] >= 0]

In [367]:
target_df.pivot_table(index='last_digit_of_day', columns='台番号', values='RB確率', aggfunc='mean', margins=True)

台番号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,All
last_digit_of_day,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,470.4,376.6,325.6,376.2,431.0,556.6,468.7,518.1,338.0,514.4,583.6,428.7,374.4,496.3,445.5,409.6,496.6,492.3,574.6,446.1,492.9,428.6,400.4,495.6,425.6,570.1,404.1,445.3,445.1,395.8,381.1,468.1,576.9,431.6,469.3,503.9,379.1,352.4,337.8,494.6,465.1,476.1,317.9,453.1,434.8,535.6,457.8,454.0,359.2,429.1,460.1,494.2,310.7,390.5,454.0,442.6,310.1,427.1,458.0,323.6,469.4,506.9,372.7,372.6,447.0,405.1,384.4,392.6,357.7,422.9,665.1,339.4,488.3,538.9,413.4,415.6,362.4,392.1,487.0,467.5,369.4,438.8
1,460.6,397.5,491.6,388.5,465.9,485.5,433.4,417.2,376.3,340.4,327.4,457.5,364.1,347.1,340.5,368.1,457.8,437.9,405.1,349.7,334.6,399.1,376.3,372.5,423.9,452.8,395.4,349.2,409.8,477.8,393.8,390.9,365.8,410.9,406.2,383.2,445.6,461.7,382.2,306.1,402.1,423.2,333.9,371.9,343.8,429.3,396.7,388.4,432.9,437.6,409.6,526.4,523.1,342.6,375.3,309.3,389.2,369.1,365.1,328.9,392.3,423.6,485.0,349.0,376.2,375.8,353.6,374.1,601.9,394.8,419.5,414.6,543.1,481.6,380.6,408.8,355.1,327.0,379.5,385.9,426.4,402.4
2,390.5,410.4,416.6,464.9,581.8,406.9,411.2,436.2,538.3,591.5,491.9,433.6,438.3,438.7,283.3,517.8,390.1,389.8,465.5,430.9,403.1,360.0,318.0,408.9,348.8,448.1,508.7,529.3,305.1,471.3,474.5,428.9,374.2,429.9,480.9,445.9,419.7,399.3,440.9,371.4,497.2,327.4,430.0,475.3,379.5,432.8,356.3,472.9,497.5,471.2,444.7,414.0,426.9,492.4,511.7,411.1,406.7,436.5,443.8,366.8,409.3,476.3,440.2,569.7,374.1,403.9,381.6,405.2,391.2,360.3,382.7,591.4,405.1,327.7,405.5,380.5,513.0,391.2,403.1,524.8,357.7,430.7
3,446.0,437.3,455.5,350.2,644.3,474.3,473.7,474.4,411.8,527.3,355.1,380.2,496.7,410.3,533.5,295.1,317.8,397.1,388.5,386.1,358.9,425.5,356.3,358.7,351.3,468.7,434.9,435.6,405.9,408.3,481.5,496.1,358.1,416.5,400.0,436.1,497.9,454.3,405.3,419.6,386.7,476.9,400.6,485.4,488.1,412.5,382.3,407.4,320.5,405.9,387.8,366.5,391.5,446.6,460.1,470.4,496.9,560.5,389.9,368.5,466.8,438.5,456.7,464.3,388.8,499.9,312.5,515.7,432.4,427.3,570.8,391.9,465.3,412.3,399.3,352.1,359.6,484.9,380.1,378.7,495.1,427.4
4,383.2,357.9,264.8,485.6,340.8,478.6,399.9,490.3,435.3,618.4,394.4,468.7,395.9,485.5,302.3,420.9,348.4,271.8,304.3,547.1,396.5,332.5,407.9,418.2,508.4,488.1,440.7,385.9,359.5,366.3,299.2,586.8,417.8,363.4,419.7,336.5,585.1,548.9,459.9,390.9,466.6,450.2,451.8,413.7,397.2,422.3,350.3,328.7,453.3,661.9,511.7,533.3,517.0,536.5,521.5,509.9,376.8,400.9,401.3,556.9,455.5,496.0,378.1,421.0,452.6,488.7,392.7,351.1,384.3,682.2,435.1,387.5,486.5,369.9,371.7,397.4,351.7,481.1,501.8,404.1,464.7,433.6
5,466.8,611.7,603.3,413.4,479.7,447.1,326.5,407.5,477.1,472.7,418.5,344.5,438.1,409.0,380.7,327.4,350.5,396.8,444.0,514.1,392.8,445.8,452.3,407.2,350.0,358.8,488.7,429.1,359.3,366.9,401.6,407.3,332.7,354.2,564.7,349.5,627.7,366.1,411.5,459.1,365.7,400.7,513.1,346.9,353.9,367.3,371.8,383.9,411.1,489.7,404.1,535.2,404.0,390.9,351.8,442.3,441.3,406.1,449.8,317.3,431.6,331.1,448.7,362.5,393.8,445.6,453.2,395.6,339.6,443.6,391.3,455.9,578.1,617.1,572.6,371.1,366.4,349.5,423.0,422.1,613.9,424.8
6,541.8,432.2,384.2,357.5,474.9,445.7,491.9,400.3,365.3,387.6,409.6,602.4,530.5,430.3,400.7,366.9,411.4,468.4,373.1,334.4,519.5,434.7,497.2,411.3,346.9,452.8,391.3,381.4,336.2,363.8,404.8,389.3,420.4,379.6,386.0,413.7,332.1,397.1,318.3,451.9,377.1,480.5,515.1,474.3,554.9,426.1,372.1,493.2,379.0,445.2,534.7,436.3,408.5,477.5,349.4,505.0,456.3,389.0,486.4,368.9,361.4,405.1,402.3,448.8,467.9,443.1,373.0,384.3,377.1,354.3,322.9,410.7,367.3,380.8,417.9,428.8,481.5,436.2,359.5,424.6,469.9,420.8
7,423.7,424.4,541.6,309.3,343

In [368]:
target_df.pivot_table(index='weekday', columns='台番号', values='RB確率', aggfunc='mean', margins=True)

台番号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,All
weekday,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,439.5,448.5,367.7,387.2,470.2,488.9,432.1,363.4,380.4,518.1,426.6,387.0,413.3,437.1,336.3,498.2,454.3,340.0,413.0,472.5,430.2,446.7,427.9,337.1,335.7,410.8,450.0,450.2,448.8,506.3,408.0,408.5,489.5,459.8,392.4,427.2,564.0,415.4,416.2,468.4,368.0,380.7,429.2,391.0,407.6,457.5,381.3,432.4,326.3,471.0,424.2,451.2,335.0,538.2,380.5,410.3,411.0,419.3,382.6,453.6,373.9,449.3,459.9,394.2,399.0,421.3,409.0,400.7,379.6,396.4,410.3,380.0,559.0,414.5,476.6,432.6,380.0,407.5,417.3,455.9,435.8,423.1
1,360.8,333.7,486.5,492.6,396.4,445.5,438.5,415.4,396.0,558.0,364.2,509.0,363.2,426.5,368.9,408.7,416.7,423.0,446.5,456.1,397.7,401.6,439.0,373.6,379.7,454.7,362.7,354.3,383.6,435.9,369.5,373.0,431.9,443.5,459.7,427.9,408.1,426.2,388.9,439.6,439.0,381.7,423.8,392.3,423.0,454.0,390.6,368.7,401.5,453.1,499.8,402.5,435.1,358.2,510.2,397.5,405.4,459.6,487.4,413.9,380.1,456.1,409.6,323.6,434.4,402.0,425.2,380.3,553.2,411.5,414.0,372.1,533.4,536.9,431.4,368.0,341.0,332.6,414.5,313.6,524.6,418.7
2,419.6,421.1,397.5,403.0,436.3,427.6,383.7,421.1,388.3,431.8,501.2,449.4,355.6,482.5,409.0,366.7,378.3,400.2,434.4,417.1,454.8,415.9,334.4,511.1,372.1,377.7,508.4,335.5,357.1,452.6,352.2,404.8,382.0,432.2,458.9,403.9,469.3,400.0,317.7,421.4,412.8,441.8,467.7,411.4,341.0,423.9,404.4,414.7,383.4,452.5,394.7,402.7,356.5,403.0,440.2,366.4,407.7,439.5,464.6,371.5,374.5,388.4,412.8,377.5,384.6,450.4,367.4,435.2,419.2,389.0,412.3,413.9,462.2,592.4,564.8,411.3,400.4,315.2,526.2,407.3,489.5,415.8
3,444.5,442.4,404.4,284.9,531.2,549.1,487.3,504.0,558.0,425.8,530.5,449.0,459.8,476.0,411.5,364.3,397.8,386.9,359.0,406.2,483.3,379.7,399.1,427.5,407.4,505.0,361.9,460.4,321.3,327.9,368.2,466.5,389.8,445.5,421.5,397.5,407.9,478.0,297.4,342.5,501.6,469.8,430.9,437.0,395.4,495.0,425.4,418.3,413.4,436.2,537.0,559.7,356.1,429.2,375.9,461.9,433.6,422.0,400.6,415.7,405.5,456.3,525.2,408.1,386.6,460.9,390.9,392.6,454.6,352.4,465.8,343.7,366.8,383.8,444.6,398.3,415.7,537.5,372.0,480.4,437.0,427.8
4,553.6,449.2,397.2,315.5,429.4,429.6,381.4,328.4,428.7,390.9,367.4,466.9,485.2,420.2,335.9,368.4,423.6,450.9,399.9,444.1,459.6,372.1,425.9,527.8,443.4,406.1,480.5,397.6,512.5,311.9,419.6,481.3,394.9,354.6,351.6,436.0,426.8,440.8,554.9,482.4,398.4,468.3,339.7,474.4,411.6,390.1,363.8,335.3,406.1,505.9,540.1,454.5,492.5,428.9,459.9,430.2,474.3,334.9,503.6,306.7,480.7,537.1,368.4,515.5,420.0,455.9,417.4,421.6,333.2,398.1,518.4,460.6,428.7,380.1,395.1,381.9,376.1,423.8,349.6,338.5,450.6,423.7
5,418.7,516.4,447.3,456.6,474.9,382.6,367.7,448.3,437.3,533.0,385.8,475.2,367.7,436.0,471.7,407.2,338.3,364.5,433.0,403.7,407.4,330.7,430.8,384.2,397.1,445.5,484.3,426.3,340.0,371.4,388.1,544.5,384.0,347.6,432.9,394.9,472.0,385.7,328.3,329.5,410.8,441.9,450.8,462.3,430.8,413.0,384.6,392.9,488.5,452.6,401.8,481.1,417.0,412.7,560.0,505.9,385.6,399.3,580.5,397.2,469.0,406.4,423.1,475.5,389.7,466.3,385.8,350.6,422.4,620.6,445.5,492.3,399.0,414.0,384.1,499.1,391.3,473.0,433.6,450.9,404.0,427.9
6,379.9,387.7,549.0,480.4,465.6,333.4,424.8,477.9,478.4,469.1,488.1,391.6,479.9,426.7,384.1,455.9,410.7,528.4,487.9,386.5,368.4,529.0,375.5,424.0,505.5,473.4,472.1,450.8,401.7,457.6,412.4,553.9,417.9,366.5,543.2,439.4,391.4,463.5,455.5,479.0,390.3,443.5,494.2,392.3,473.1,401.0,391.1,489.6,436.2,410.0,413.7,419.1,490.1,512.0,434.4,411.8,399.6,428.6,386.4,441.7,423.0,447.1,502.6,396.8,469.9,494.7,367.6,416.4,372.5,421.6,496.9,489.1,443.8,377.0,375.5,337.9,410.7,378.5,436.2,405.3,462.3,437.7
All,429.4,427.6,436.1,403.7,458.0,437.1

In [382]:
target_df.pivot_table(index='day', columns='台番号', values='RB確率', aggfunc='mean', margins=True).iloc[28, :]

台番号
1001   347.3
1002   362.3
1003   359.2
1004   410.7
1005   398.7
1006   299.0
1007   303.5
1008   403.0
1009   404.0
1010   279.7
1011   416.0
1012   588.0
1013   334.2
1014   523.0
1015   268.3
1016   443.2
1017   310.2
1018   567.8
1019   311.0
1020   294.2
1021   478.0
1022   401.0
1023   331.0
1024   702.5
1025   637.2
1026   370.5
1027   361.5
1028   407.5
1029   546.2
1030   312.2
1031   393.8
1032   493.3
1033   503.5
1034   407.8
1035   453.7
1036   391.0
1037   351.0
1038   293.0
1039   392.5
1040   339.3
1041   318.5
1042   443.0
1043   345.5
1044   475.0
1045   341.0
1046   447.2
1047   609.2
1048   261.2
1049   521.5
1050   302.7
1069   352.8
1070   347.8
1071   266.8
1072   355.3
1073   711.8
1074   511.3
1075   402.7
1076   366.8
1077   299.0
1078   280.3
1079   344.8
1080   398.2
1081   532.8
1082   330.7
1083   488.5
1084   358.7
1085   401.7
1086   499.0
1087   288.8
1088   340.5
1089   406.8
1090   590.0
1091   375.3
1092   419.7
1093   457.7
1094   358.8
1095   3

In [378]:
df[(df["台番号"] == 1019) & (df["day"] == 29)].describe()

,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date,month,weekday,day,last_digit_of_day
count,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6,6.0,6.0,6.0,6.0
mean,1019.0,5970.5,1773.5,25.7,23.0,142.5,268.0,311.0,2024-11-13 12:00:00,8.5,3.7,29.0,9.0
min,1019.0,1202.0,156.0,3.0,3.0,106.0,191.0,221.0,2024-08-29 00:00:00,1.0,1.0,29.0,9.0
25%,1019.0,4269.2,502.0,15.2,10.5,113.5,228.0,234.8,2024-10-06 12:00:00,8.2,2.2,29.0,9.0
50%,1019.0,6362.5,1198.5,26.0,25.5,130.0,251.5,277.5,2024-11-13 12:00:00,9.5,3.5,29.0,9.0
75%,1019.0,8307.2,2444.0,35.2,36.0,168.2,283.2,378.8,2024-12-21 12:00:00,10.8,5.5,29.0,9.0
max,1019.0,9369.0,4959.0,49.0,39.0,200.0,400.0,457.0,2025-01-29 00:00:00,12.0,6.0,29.0,9.0
std,0.0,3140.2,1827.4,16.7,15.8,38.4,73.1,98.5,NaN,3.9,2.1,0.0,0.0


In [339]:
target_df = df[(df['G数'] >= 0) & (df['date'] >= pd.to_datetime("2024-11-01"))]
df_rb_rate = target_df.pivot_table(index='台番号', columns='date', values='RB確率', aggfunc='mean')

pd.concat([df_rb_rate.iloc[:, 84:89], df_rb_rate.iloc[:, 130:135]], axis=1)

date,2025-01-25,2025-01-26,2025-01-27,2025-01-28,2025-01-29,2025-03-14,2025-03-15,2025-03-16,2025-03-17,2025-03-18
台番号,,,,,,,,,,
1001,465.0,606.0,496.0,230.0,284.0,413.0,302.0,458.0,389.0,315.0
1002,338.0,433.0,389.0,323.0,361.0,1003.0,1158.0,305.0,272.0,437.0
1003,463.0,478.0,227.0,295.0,580.0,273.0,229.0,372.0,511.0,366.0
1004,412.0,322.0,396.0,0.0,471.0,0.0,407.0,321.0,0.0,500.0
1005,396.0,332.0,392.0,462.0,509.0,355.0,698.0,357.0,272.0,345.0
1006,284.0,351.0,316.0,267.0,0.0,441.0,258.0,379.0,784.0,228.0
1007,220.0,575.0,451.0,262.0,237.0,232.0,311.0,317.0,379.0,0.0
1008,215.0,222.0,0.0,336.0,356.0,416.0,359.0,311.0,422.0,336.0
1009,622.0,281.0,268.0,823.0,0.0,441.0,289.0,533.0,433.0,239.0
